In [36]:
import os
import numpy as np
import cv2
from numpy import load
from numpy import save
import gc 

In [37]:
#設定資料路徑
path = "D:\\tensorflow\Covid_data"
os.chdir(path)

In [38]:
labels = os.listdir(path)
labels

['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

In [39]:
#定義合併像素的函數.
def merge(data,x,y):
    n = np.shape(data)[0]
    ylength = np.shape(data)[1]
    xlength = np.shape(data)[2]
    gc.collect()
    if(x == 1 and y == 1):
        return data
    new = np.zeros((n,ylength + y - 1,xlength + x - 1))
    for i in range(0,y):
        for j in range(0,x):
            new[:,i:ylength + i,j:xlength + j] += data
    new = new/(x*y)
    gc.collect()
    normal = np.zeros((n,int((ylength + y - 1)/y),int((xlength + x - 1)/x)))
    normal[:,:int(ylength/y),:int(xlength/x)] = new[:,y - 1:ylength:y,x - 1:xlength:x]
    if(ylength%y != 0):
        normal[:,np.shape(normal)[1] - 1,:int(xlength/x)] = new[:,ylength + y - 1 - ylength%y,x - 1:xlength:x]/(ylength%y)*y
    if(xlength%x != 0):
        normal[:,:int(ylength/y),np.shape(normal)[2] - 1] = new[:,y - 1:ylength:y,xlength + x - 1 - xlength%x]/(xlength%x)*x
    if((ylength%y != 0) and ((xlength%x != 0))):
        normal[:,np.shape(normal)[1]-1,np.shape(normal)[2]-1] = new[:,ylength + y - 1 - ylength%y,xlength + x - 1 - xlength%x]/(ylength%y)/(xlength%x)*y*x
    return normal

In [40]:
number = []
mpath = "D:\\tensorflow\merge_pic"
for i in range(len(labels)):
    label_path = path + "/" + labels[i]
    number += [len(os.listdir(label_path))]
for i in range(len(labels)):
    label_path = path + "/" + labels[i]
    os.chdir(label_path)
    pic_list = os.listdir(label_path)
    data = np.zeros((number[i],299,299),dtype = int)
    for j in range(len(pic_list)):
        data[j,:,:] = cv2.imread(pic_list[j], cv2.IMREAD_GRAYSCALE)
    new_data = merge(data,2,2)
    print(np.shape(new_data))
    new_data = new_data
    del data
    gc.collect()
    new_data = new_data.astype(int)
    gc.collect()
    os.chdir(mpath)
    os.mkdir(labels[i])
    picpath = mpath + "\\" + labels[i]
    os.chdir(picpath)
    for j in range(number[i]):
        cv2.imwrite("%d.png"%(j), new_data[j,:,:])
    del new_data
    gc.collect()

(3616, 150, 150)
(6012, 150, 150)
(10192, 150, 150)
(1345, 150, 150)
